In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from numpy import linalg
from random import random
from copy import deepcopy
from scipy.linalg import hankel, eigh
from matplotlib import style
from matplotlib.patches import Ellipse

In [2]:
df_n = pd.read_csv('~/data/WADI_14days_2017.csv')
df_n

,Row,Date,Time,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_002_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_003_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_004_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_005_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_FIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_LS_001_AL,...,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_MV_001_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_MV_002_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_MV_003_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_001_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_002_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_003_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_004_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\LEAK_DIFF_PRESSURE,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\PLANT_START_STOP_LOG,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\TOTAL_CONS_REQUIRED_FLOW
0,1,9/25/2017,6:00:00 PM,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,...,1,1,1,1,1,1,1,67.9651,1,0.68
1,2,9/25/2017,6:00:01 PM,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,...,1,1,1,1,1,1,1,67.9651,1,0.68
2,3,9/25/2017,6:00:02 PM,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,...,1,1,1,1,1,1,1,67.9651,1,0.68
3,4,9/25/2017,6:00:03 PM,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,...,1,1,1,1,1,1,1,67.1948,1,0.68
4,5,9/25/2017,6:00:04 PM,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,...,1,1,1,1,1,1,1,67.1948,1,0.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048566,1048567,10/07/17,9:16:06 PM,175.855,0.589478,11.8941,479.191,0.331571,0.001128,0,...,1,1,1,1,1,1,1,60.6305,1,0.25
1048567,1048568,10/07/17,9:16:07 PM,175.855,0.589478,11.8941,479.191,0.331571,0.001128,0,...,1,1,1,1,1,1,1,60.6305,1,0.25
1048568,1048569,10/07/17,9:16:08 PM,175.855,0.589478,11.8941,479.191,0.331571,0.001128,0,...,1,1,1,1,1,1,1,60.6305,1,0.25
1048569,1048570,10/07/17,9:16:09 PM,175.896,0.613476,11.8913,479.224,0.331622,0.001173,0,...,1,1,1,1,1,1,1,60.4477,1,0.25


In [3]:
for col in df_n.columns:
  df_n[col].fillna(df_n[col].mode()[0],inplace=True)
# df_n.isnull().sum().unique()

In [4]:
df_n=df_n.iloc[:,3:]

In [5]:
df_n['Attack?']=1

In [6]:
data = []
for i in range(0,len(df_n),3600):
  data.append(df_n[i:i+3600].to_numpy())

In [7]:
data = data[:-1]
data = np.asarray(data)

In [8]:
data.shape

(291, 3600, 124)

In [9]:
df_a=pd.read_csv('~/data/WADI_attackdataLABELED.csv')

In [10]:
df_a.isnull().sum()

Row                                      2
Date                                     2
Time                                     2
1_AIT_001_PV                             2
1_AIT_002_PV                             2
                                        ..
3_P_004_STATUS                           2
LEAK_DIFF_PRESSURE                       2
PLANT_START_STOP_LOG                     2
TOTAL_CONS_REQUIRED_FLOW                 2
Attack LABLE (1:No Attack, -1:Attack)    0
Length: 127, dtype: int64

In [11]:
df_a=df_a[:-3]
df_a.isnull().sum().unique()

array([0])

In [12]:
df_a=df_a.iloc[:,3:]

In [13]:
data_a = []
for i in range(0,len(df_a),3600):
  data_a.append(df_a[i:i+3600].to_numpy())
  
data_a = np.asarray(data_a)

In [14]:
# STRONG SUPERVISION
ss_normal_a = []
ss_attack_a = []
for hour_data in data_a:
    if np.any(hour_data[:, -1] == -1):
        ss_attack_a.append(hour_data)
    else:
        ss_normal_a.append(hour_data)
ss_normal_a = np.asarray(ss_normal_a)
ss_attack_a = np.asarray(ss_attack_a)

In [15]:
ss_normal_a.shape

(37, 3600, 124)

In [16]:
ss_attack_a.shape

(11, 3600, 124)

In [17]:
ss_normal = np.concatenate((data,ss_normal_a))
ss_normal.shape

(328, 3600, 124)

In [18]:
ss_attack = ss_attack_a
ss_attack.shape

(11, 3600, 124)

In [19]:
np.random.shuffle(ss_normal)
normal_train, normal_test = ss_normal[:261], ss_normal[261:]

In [20]:
np.random.shuffle(ss_attack)
attack_train,attack_test=ss_attack_a[:9],ss_attack_a[9:]

In [21]:
#training data
train_data=np.concatenate((normal_train,attack_train))
#test data
test_data=np.concatenate((normal_test,attack_test))

In [22]:
test_data.shape

(69, 3600, 124)

In [23]:
train_data.shape

(270, 3600, 124)

In [24]:
with open('traindata_ss.npy','wb') as f:
    np.save(f,train_data)

In [25]:
with open('testdata_ss.npy','wb') as f:
    np.save(f,test_data)